<a href="https://colab.research.google.com/github/Douboo/tf_env_debug/blob/master/custom_layers_and_model_subclassing_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorflow==2.0.0

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.layers import Input, Dense, Conv2D, BatchNormalization, Activation, Dropout, GRU, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras import activations
from tensorflow.keras.layers import Layer
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras.layers import GRU, concatenate, Lambda

In [0]:
ENCODER_SEQ_LEN = 30
DECODER_SEQ_LEN = 20
VOCAB_SIZE = 500
units = 16

In [0]:
tf.__version__

'2.0.0'

In [0]:
tf.keras.backend.clear_session()
class Encoder(tf.keras.layers.Layer):
    def __init__(self, vocab_size, embedding_dim, input_length, units):
        super(Encoder, self).__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.input_length = input_length
        self.units = units
        self.embedding = Embedding(input_dim=VOCAB_SIZE, output_dim=50, input_length=self.input_length,
                           mask_zero=False, name="embedding_layer_encoder")
        self.gru = GRU(self.units, return_state=True, return_sequences=True, name="Encoder_GRU")
    @tf.function
    def call(self, inputs, training=True):
        x_embedd = self.embedding(inputs)
        gru_output, gru_state = self.gru(x_embedd)
        return gru_output, gru_state
    
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)
  @tf.function
  def call(self, query, values):
    # hidden shape == (batch_size, hidden size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden size)
    # we are doing this to perform addition to calculate the score
    hidden_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(values) + self.W2(hidden_with_time_axis)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector

class onestepDecoder(tf.keras.layers.Layer):
    def __init__(self, vocab_size, embedding_dim, dec_units, att_units):
        super(onestepDecoder, self).__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.dec_units = dec_units
        self.att_units = att_units
        self.embedd = Embedding(input_dim=self.vocab_size, output_dim=self.embedding_dim,
                      input_length=1, mask_zero=False, name="Decoder_Embedding_layer")
        self.att_layer = BahdanauAttention(units=self.att_units) #name='Attention')
        self.dense = Dense(self.vocab_size, activation="softmax", name="DenseOut")
        self.gru = GRU(units=self.dec_units, return_state=True, name="DecGRU")
    @tf.function
    def call(self, input_decoder, input_state, encoder_outputs, training=True):
        x_embedd = self.embedd(input_decoder)
        context_vector = self.att_layer(input_state, encoder_outputs )
        concat = tf.concat([tf.expand_dims(context_vector, 1), x_embedd], axis=-1)
        decoder_output, Decoder_state = self.gru(concat, initial_state=input_state)
        output = self.dense(decoder_output)
        return (output, Decoder_state)
class Decoder(tf.keras.layers.Layer):
    def __init__(self, vocab_size, embedding_dim, dec_units, att_units):
        super(Decoder, self).__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.dec_units = dec_units
        self.att_units = att_units
        self.stepdec = onestepDecoder(self.vocab_size, self.embedding_dim, self.dec_units, self.att_units)
    @tf.function
    def call(self, input_decoder, input_state, encoder_outputs):
        all_outputs= tf.TensorArray(tf.float32, size=input_decoder.shape[1], name="output_arrays")
        for timestep in range(input_decoder.shape[1]):
            output, input_state = self.stepdec(input_decoder[:,timestep:timestep+1], input_state, encoder_outputs)
            all_outputs = all_outputs.write(timestep, output)
        all_outputs = tf.transpose(all_outputs.stack(), [1, 0, 2])
        return all_outputs

In [0]:
class ED(Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, dec_units, att_units, input_length):
        super(ED, self).__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.dec_units = dec_units
        self.att_units = att_units
        self.enc_units = enc_units
        self.input_length = input_length
        self.encoder = Encoder(vocab_size=self.vocab_size, embedding_dim=self.embedding_dim, 
                               input_length=self.input_length, units=self.enc_units)
        self.decoder = Decoder(vocab_size=self.vocab_size, embedding_dim=self.embedding_dim, 
                               dec_units=self.dec_units, att_units=self.att_units)
    @tf.function
    def call(self, inputs):
        encoder_input, decoder_input = inputs
        x_gru_out, x_gru_state = self.encoder(encoder_input)
        dec_out = self.decoder(decoder_input, x_gru_state, x_gru_out)
        return dec_out

In [0]:
encoder_decoder = ED(vocab_size=VOCAB_SIZE, embedding_dim=50, enc_units=16, 
                     dec_units=16, att_units=20, input_length=ENCODER_SEQ_LEN)

In [0]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [0]:
x = np.random.randint(0, 499, size=(2000, ENCODER_SEQ_LEN))
y = np.random.randint(0, 499, size=(2000, DECODER_SEQ_LEN))

In [0]:
train_loss = tf.keras.metrics.Mean(name='train_loss')

In [0]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = encoder_decoder(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, encoder_decoder.trainable_variables)
  optimizer.apply_gradients(zip(gradients, encoder_decoder.trainable_variables))
  train_loss(loss)

In [0]:
train_ds = tf.data.Dataset.from_tensor_slices(
    ((x,y), y)).shuffle(1000).batch(32)

In [0]:
train_ds

<BatchDataset shapes: (((None, 30), (None, 20)), (None, 20)), types: ((tf.int64, tf.int64), tf.int64)>

In [0]:
import datetime
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)

In [0]:
EPOCHS = 5
tf.summary.trace_on(graph=True, profiler=True)
for epoch in range(EPOCHS):
  for x_temp, y_temp in train_ds:
    train_step(x_temp, y_temp)
  with train_summary_writer.as_default():
    tf.summary.scalar('loss', train_loss.result(), step=epoch)

  template = 'Epoch {}, Loss: {}'
  print(template.format(epoch+1, train_loss.result()))
  train_loss.reset_states()
with train_summary_writer.as_default():
  tf.summary.trace_export(
      name="my_func_trace",
      step=0,
      profiler_outdir='logs/gradient_tape/')

Epoch 1, Loss: 2.3444130420684814
Epoch 2, Loss: 1.8981623649597168
Epoch 3, Loss: 1.5713832378387451
Epoch 4, Loss: 1.3242261409759521
